In [12]:
# =========================
# SILENCE TF LOGS
# =========================
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# =========================
# GPU SETUP (AUTOMATIC)
# =========================
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("✅ GPU detected")
    except:
        print("⚠️ GPU detected but memory growth failed")
else:
    print("⚠️ No GPU detected, running on CPU")

# =========================
# IMPORTS
# =========================
import cv2
import numpy as np
from tqdm.auto import tqdm

from tensorflow.keras.layers import (
    Input, Conv2D, TimeDistributed, ConvLSTM2D
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback

# =========================
# CONFIGURATION
# =========================
TIME_STEPS = 5
IMG_SIZE = (128, 128)
CHANNELS = 1

BATCH_SIZE = 8
EPOCHS = 20
LEARNING_RATE = 1e-4

UCSD_TRAIN_PATH = "../datasets/UCSDped2/train"  

# =========================
# DATA LOADING
# =========================
def load_frames_from_folder(folder_path):
    frames = []
    for file in sorted(os.listdir(folder_path)):
        if file.lower().endswith(".tif"):
            img = cv2.imread(
                os.path.join(folder_path, file),
                cv2.IMREAD_GRAYSCALE
            )
            img = cv2.resize(img, IMG_SIZE)
            img = img.astype("float32") / 255.0
            frames.append(img)
    return np.array(frames)


def create_sequences(frames, time_steps):
    X, Y = [], []
    for i in range(len(frames) - time_steps):
        X.append(frames[i:i + time_steps])
        Y.append(frames[i + time_steps])
    return np.array(X), np.array(Y)


def load_ucsd_training_data(base_path):
    all_X, all_Y = [], []

    for folder in sorted(os.listdir(base_path)):
        folder_path = os.path.join(base_path, folder)
        if os.path.isdir(folder_path):
            frames = load_frames_from_folder(folder_path)
            if len(frames) <= TIME_STEPS:
                continue
            X, Y = create_sequences(frames, TIME_STEPS)
            all_X.append(X)
            all_Y.append(Y)

    X = np.concatenate(all_X, axis=0)[..., np.newaxis]
    Y = np.concatenate(all_Y, axis=0)[..., np.newaxis]

    return X, Y

# =========================
# MODEL
# =========================
def build_model():
    inputs = Input(
        shape=(TIME_STEPS, IMG_SIZE[0], IMG_SIZE[1], CHANNELS)
    )

    x = TimeDistributed(
        Conv2D(32, 3, strides=2, padding="same", activation="relu")
    )(inputs)

    x = TimeDistributed(
        Conv2D(64, 3, strides=2, padding="same", activation="relu")
    )(x)

    x = TimeDistributed(
        Conv2D(128, 3, strides=2, padding="same", activation="relu")
    )(x)

    x = ConvLSTM2D(
        128, 3, padding="same",
        return_sequences=True,
        activation="tanh"
    )(x)

    x = ConvLSTM2D(
        128, 3, padding="same",
        return_sequences=False,
        activation="tanh"
    )(x)

    outputs = Conv2D(
        128, 3, padding="same", activation="linear"
    )(x)

    return Model(inputs, outputs)

# =========================
# TQDM CALLBACK
# =========================
class TQDMProgressBar(Callback):
    def on_train_begin(self, logs=None):
        self.epochs = self.params["epochs"]

    def on_epoch_begin(self, epoch, logs=None):
        self.bar = tqdm(
            total=self.params["steps"],
            desc=f"Epoch {epoch + 1}/{self.epochs}",
            leave=True,
            dynamic_ncols=True
        )

    def on_batch_end(self, batch, logs=None):
        self.bar.update(1)
        self.bar.set_postfix(
            loss=f"{logs['loss']:.6f}"
        )

    def on_epoch_end(self, epoch, logs=None):
        self.bar.close()

# =========================
# TRAINING
# =========================
print("📥 Loading UCSD Ped2 training data...")
X_train, Y_train = load_ucsd_training_data(UCSD_TRAIN_PATH)

# Downsample target to latent resolution (128 → 16)
Y_train = tf.image.resize(Y_train, (16, 16))

model = build_model()
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss="mse"
)

model.summary()

print("🚀 Training started...")
history = model.fit(
    X_train,
    Y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    shuffle=True,
    verbose=0,                      # IMPORTANT
    callbacks=[TQDMProgressBar()]
)

print("✅ Training completed")

# =========================
# SAVE MODEL
# =========================
model.save("convlstm_ucsd_model.h5")
print("💾 Model saved successfully")


⚠️ No GPU detected, running on CPU
📥 Loading UCSD Ped2 training data...


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)           │ (None, 5, 128, 128, 1)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_18                  │ (None, 5, 64, 64, 32)       │             320 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_19                  │ (None, 5, 32, 32, 64)       │          18,496 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_20                  │ (None, 5, 16, 16, 128)      │          73,856 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_lstm2d_12 (ConvLSTM2D)          │ (None, 5, 16, 16, 128)      │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_lstm2d_13 (ConvLSTM2D)          │ (None, 16, 16, 128)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_27 (Conv2D)                   │ (None, 16, 16, 128)         │         147,584 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,600,576 (9.92 MB)

 Trainable params: 2,600,576 (9.92 MB)

 Non-trainable params: 0 (0.00 B)

🚀 Training started...


Epoch 20/20: 100%|████████████████████████████████████████████████████| 309/309 [12:21<00:00,  2.40s/it, loss=0.000085]


✅ Training completed
💾 Model saved successfully


In [19]:
# =====================================================
# GPU SETUP (AUTOMATIC)
# =====================================================
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    print("✅ GPU detected and enabled")
else:
    print("⚠️ No GPU detected, running on CPU")

# =====================================================
# IMPORTS
# =====================================================
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_auc_score

# =====================================================
# CONFIGURATION
# =====================================================
TIME_STEPS = 5
IMG_SIZE = (128, 128)

UCSD_TEST_PATH = "../datasets/UCSDped2/test"
UCSD_GT_PATH   = "../datasets/UCSDped2/test_gt"
MODEL_PATH = "convlstm_ucsd_model.h5"

# =====================================================
# LOAD MODEL
# =====================================================
model = load_model(MODEL_PATH, compile=False)
print("✅ Model loaded")

# =====================================================
# DATA UTILITIES
# =====================================================
def load_frames_from_folder(folder_path):
    frames = []
    for file in sorted(os.listdir(folder_path)):
        if file.endswith(".tif"):
            img = cv2.imread(os.path.join(folder_path, file), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, IMG_SIZE)
            img = img.astype("float32") / 255.0
            frames.append(img)
    return np.array(frames)

def create_sequences(frames, time_steps):
    X, Y = [], []
    for i in range(len(frames) - time_steps):
        X.append(frames[i:i + time_steps])
        Y.append(frames[i + time_steps])
    X = np.array(X)[..., np.newaxis]
    Y = np.array(Y)[..., np.newaxis]
    return X, Y

# =====================================================
# GROUND TRUTH UTILITIES
# =====================================================
def load_frame_level_gt(gt_folder):
    """
    Convert pixel-level GT to frame-level labels
    """
    labels = []
    for file in sorted(os.listdir(gt_folder)):
        if file.endswith(".bmp"):
            gt = cv2.imread(os.path.join(gt_folder, file), cv2.IMREAD_GRAYSCALE)
            label = 1 if np.max(gt) > 0 else 0
            labels.append(label)
    return np.array(labels)

# =====================================================
# ANOMALY SCORE COMPUTATION
# =====================================================
def compute_anomaly_scores(frames):
    X, Y = create_sequences(frames, TIME_STEPS)

    # Downsample target to latent size (16x16)
    Y_ds = tf.image.resize(Y, (16, 16))

    preds = model.predict(X, batch_size=8, verbose=0)

    scores = np.mean((preds - Y_ds.numpy()) ** 2, axis=(1, 2, 3))
    return scores

def dynamic_threshold(scores):
    return np.mean(scores) + 2 * np.std(scores)

# =====================================================
# RUN TESTING + AUC
# =====================================================
print("\n📊 Running anomaly detection on UCSD Ped2 TEST videos...\n")

all_scores = []
all_labels = []

for folder in sorted(os.listdir(UCSD_TEST_PATH)):
    test_path = os.path.join(UCSD_TEST_PATH, folder)
    gt_path = os.path.join(UCSD_GT_PATH, folder + "_gt")

    if not os.path.isdir(test_path):
        continue

    frames = load_frames_from_folder(test_path)
    scores = compute_anomaly_scores(frames)

    # Load GT labels
    gt_labels = load_frame_level_gt(gt_path)

    # Align GT with scores (skip first TIME_STEPS frames)
    gt_labels = gt_labels[TIME_STEPS:]

    th = dynamic_threshold(scores)
    anomalies = scores > th

    print(f"{folder}")
    print(f"  Frames tested : {len(scores)}")
    print(f"  Score range   : {scores.min():.6f} → {scores.max():.6f}")
    print(f"  Threshold    : {th:.6f}")
    print(f"  Anomalies    : {np.sum(anomalies)} frames\n")

    all_scores.extend(scores.tolist())
    all_labels.extend(gt_labels.tolist())

# =====================================================
# AUC COMPUTATION
# =====================================================
auc = roc_auc_score(all_labels, all_scores)
print("🎯 UCSD Ped2 Frame-level AUC:", round(auc, 4))
print("✅ Testing + AUC completed")


⚠️ No GPU detected, running on CPU
✅ Model loaded

📊 Running anomaly detection on UCSD Ped2 TEST videos...

Test001
  Frames tested : 175
  Score range   : 0.000059 → 0.000428
  Threshold    : 0.000268
  Anomalies    : 6 frames

Test002
  Frames tested : 175
  Score range   : 0.000039 → 0.000535
  Threshold    : 0.000346
  Anomalies    : 10 frames

Test003
  Frames tested : 145
  Score range   : 0.000047 → 0.000315
  Threshold    : 0.000282
  Anomalies    : 5 frames

Test004
  Frames tested : 175
  Score range   : 0.000016 → 0.000608
  Threshold    : 0.000443
  Anomalies    : 8 frames

Test005
  Frames tested : 145
  Score range   : 0.000066 → 0.000443
  Threshold    : 0.000333
  Anomalies    : 5 frames

Test006
  Frames tested : 175
  Score range   : 0.000034 → 0.000363
  Threshold    : 0.000283
  Anomalies    : 8 frames

Test007
  Frames tested : 175
  Score range   : 0.000034 → 0.000328
  Threshold    : 0.000224
  Anomalies    : 4 frames

Test008
  Frames tested : 175
  Score range 